In [1]:
import numpy as np 
import pandas as pd
import os
import matplotlib.pyplot as plt
from PIL import Image
import torch
import time
import wandb
wandb.login(key="f659082c2b19bf3ffaaceceb36c1e280541f6b11")

wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: yeshu183 (yeshu183-indian-institute-of-technology-madras) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

In [2]:
base_path_train = "/kaggle/input/naturalist/inaturalist_12K/train" 
input_shape = (224,224)
id2label = {}
label2id = {}
label_list = []
X = []
y = []
for id,label in enumerate(os.listdir(base_path_train)):
    # if label[0]==".":
    #     continue
    id2label[id] = label
    label2id[label] = id
    label_list.append(label)
    label_path = os.path.join(base_path_train,label)
    for img_path in os.listdir(label_path):
        # if img_path == ".DS_Store":
        #     continue
        img = np.array(Image.open(os.path.join(label_path,img_path)))
        img.resize((*input_shape,3))
        X.append(img)
        y.append(id)
    print(f"id:{id}, Label: {label} done")
X = np.array(X).transpose(0, 3, 1, 2)
X = torch.tensor(X, dtype=torch.float32)
y = torch.tensor(y, dtype=torch.long)
print(X.shape)
print(y.shape)

id:0, Label: Reptilia done
id:1, Label: Animalia done
id:2, Label: Arachnida done
id:3, Label: Amphibia done
id:4, Label: Aves done
id:5, Label: Mollusca done
id:6, Label: Fungi done
id:7, Label: Insecta done
id:8, Label: Plantae done
id:9, Label: Mammalia done
torch.Size([9999, 3, 224, 224])
torch.Size([9999])


In [3]:
base_path_test = "/kaggle/input/naturalist/inaturalist_12K/val" 
input_shape = (224,224)
X_test = []
y_test = []
for id,label in enumerate(os.listdir(base_path_test)):
    # if label[0]==".":
    #     continue
    label_path = os.path.join(base_path_test,label)
    for img_path in os.listdir(label_path):
        # if img_path == ".DS_Store":
        #     continue
        img = np.array(Image.open(os.path.join(label_path,img_path)))
        img.resize((*input_shape,3))
        X_test.append(img)
        y_test.append(id)
    print(f"Test data id:{id}, Label: {label} done")
X_test = np.array(X_test).transpose(0, 3, 1, 2)
X_test = torch.tensor(X_test, dtype=torch.float32)
y_test = torch.tensor(y_test, dtype=torch.long)
print(X_test.shape)
print(y_test.shape)

Test data id:0, Label: Reptilia done
Test data id:1, Label: Animalia done
Test data id:2, Label: Arachnida done
Test data id:3, Label: Amphibia done
Test data id:4, Label: Aves done
Test data id:5, Label: Mollusca done
Test data id:6, Label: Fungi done
Test data id:7, Label: Insecta done
Test data id:8, Label: Plantae done
Test data id:9, Label: Mammalia done
torch.Size([2000, 3, 224, 224])
torch.Size([2000])


In [4]:
print(id2label)
print(label2id)
print(label_list)

{0: 'Reptilia', 1: 'Animalia', 2: 'Arachnida', 3: 'Amphibia', 4: 'Aves', 5: 'Mollusca', 6: 'Fungi', 7: 'Insecta', 8: 'Plantae', 9: 'Mammalia'}
{'Reptilia': 0, 'Animalia': 1, 'Arachnida': 2, 'Amphibia': 3, 'Aves': 4, 'Mollusca': 5, 'Fungi': 6, 'Insecta': 7, 'Plantae': 8, 'Mammalia': 9}
['Reptilia', 'Animalia', 'Arachnida', 'Amphibia', 'Aves', 'Mollusca', 'Fungi', 'Insecta', 'Plantae', 'Mammalia']


## Part-A

In [5]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchvision
from torchvision import datasets, models, transforms
from torchsummary import summary

In [6]:
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
X_train,X_val,y_train,y_val = train_test_split(X,y,test_size=0.2,random_state=42)
#Getting (X,y) pairs
train_data = TensorDataset(X_train, y_train)
val_data = TensorDataset(X_val, y_val)
test_data = TensorDataset(X_test, y_test)

In [7]:
class customCNN(nn.Module):
    def __init__(self,m=16,k=3,n=256,m_factor=2,n_conv=5,activation_func="ReLU",batch_norm=True,dropout=0.5):
        super().__init__()
        self.k = k # size of filters (kxk)
        self.n = n # no.of neurons in the dense layer
        self.m = m # no. of filters
        self.m_fac = m_factor # filter multiplier
        self.conv_layers = nn.ModuleList()
        self.batch_norm = batch_norm
        self.bn_layers = nn.ModuleList()
        self.dropout = nn.Dropout(dropout)
        self.n_conv = n_conv
        m = self.m
        for i in range(self.n_conv):
            if i==0:
                self.conv_layers.append(nn.Conv2d(3,int(m),self.k))
                self.bn_layers.append(nn.BatchNorm2d(int(m)))
            else:
                if m*self.m_fac>=1:
                    self.conv_layers.append(nn.Conv2d(int(m),int(m*self.m_fac),self.k))
                    self.bn_layers.append(nn.BatchNorm2d(int(m*self.m_fac)))
                    m = m*self.m_fac
                else:
                    self.m_fac = 1
                    self.conv_layers.append(nn.Conv2d(int(m),int(m*self.m_fac),self.k))
                    self.bn_layers.append(nn.BatchNorm2d(int(m*self.m_fac)))
                    m = m*self.m_fac
            
        self.maxpool = nn.MaxPool2d(2,2)
        self.fc1 = nn.LazyLinear(self.n)
        self.fc2 = nn.Linear(self.n, 10)
        activations = {
            "mish": nn.Mish(),
            "gelu": nn.GELU(),
            "silu": nn.SiLU(),
            "relu": nn.ReLU()
        }
        self.activation = activations.get(activation_func.lower(), nn.ReLU())
    def forward(self,x):
        for i in range(self.n_conv):
            x = self.conv_layers[i](x)
            if self.batch_norm:
                x = self.bn_layers[i](x)
            x = self.activation(x)
            x = self.maxpool(x)
        x = torch.flatten(x,1)
        x = self.fc1(x)
        x = self.dropout(x)
        x = self.activation(x)
        x = self.fc2(x)
        # output = F.softmax(x,dim=1)
        return x

In [8]:
def data_loading(data,train=True,batch_size = 64):
    data_loader = DataLoader(train_data, batch_size=batch_size, shuffle=train)
    return data_loader

In [9]:
def train(train_loader,val_loader,n_epochs=10):
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=0.01)
    
    train_losses = []
    train_accuracies = []
    val_losses = []
    val_accuracies = []
    
    for epoch in range(n_epochs):
        start_time = time.time()
    
        model.train()
        running_loss = 0.0
        correct = 0
        total = 0
    
        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)
    
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
    
            running_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    
        train_loss = running_loss / len(train_loader)
        train_acc = 100 * correct / total
        train_losses.append(train_loss)
        train_accuracies.append(train_acc)
    
        # Validation
        model.eval()
        val_loss = 0.0
        val_correct = 0
        val_total = 0
    
        with torch.no_grad():
            for inputs, labels in val_loader:
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = model(inputs)
                loss = criterion(outputs, labels)
                val_loss += loss.item()
                _, predicted = torch.max(outputs.data, 1)
                val_total += labels.size(0)
                val_correct += (predicted == labels).sum().item()
    
        val_loss /= len(val_loader)
        val_acc = 100 * val_correct / val_total
        val_losses.append(val_loss)
        val_accuracies.append(val_acc)
    
        end_time = time.time()
        epoch_time = end_time - start_time
    
        print(f"Epoch {epoch+1} | Time: {epoch_time:.2f}s | "
              f"Train Loss: {train_loss:.3f}, Train Acc: {train_acc:.2f}% | "
              f"Val Loss: {val_loss:.3f}, Val Acc: {val_acc:.2f}%")
    
    return train_losses,train_accuracies,val_losses,val_accuracies

In [10]:
sweep_config = {
    'method': 'bayes',
    'metric': {
        'name': 'test_accuracy',
        'goal': 'maximize'
    },
    'parameters': {
        'm': {'values': [32, 64]},
        'm_factor': {'values': [1, 2, 0.5]},
        'dropout': {'values': [0.2, 0.3]},
        'activation_func': {'values': ['ReLU', 'GELU', 'SiLU', 'Mish']},
        'batch_norm': {'values': [True, False]},
        'batch_size': {'values': [32, 64]},
        'data_augmentation': {'values': [True, False]}
    }
}


In [14]:
def train_sweep():
    wandb.init()
    config = wandb.config

    # Set run name based on hyperparameters
    run_name = (f"act_{config.activation_func}_bs_{config.batch_size}_m_{config.m}_"
                f"mf_{config.m_factor}_do_{config.dropout}_bn_{config.batch_norm}_aug_{config.data_augmentation}")
    wandb.run.name = run_name

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = customCNN(
        m=config.m,
        m_factor=config.m_factor,
        activation_func=config.activation_func,
        batch_norm=config.batch_norm,
        dropout=config.dropout
    ).to(device)

    # Data augmentation transforms (if needed)
    transform_train = transforms.Compose([
        transforms.RandomHorizontalFlip(),
        transforms.RandomCrop(32, padding=4),
        transforms.ToTensor()
    ]) if config.data_augmentation else transforms.ToTensor()
    transform_val = transforms.ToTensor()

    # Assuming train_data, val_data, test_data are already pre-loaded tensors/datasets.
    train_loader = DataLoader(train_data, batch_size=config.batch_size, shuffle=True)
    val_loader = DataLoader(val_data, batch_size=config.batch_size, shuffle=False)
    test_loader = DataLoader(test_data, batch_size=config.batch_size, shuffle=False)

    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=0.001)

    for epoch in range(15):  # Adjust the number of epochs if needed
        epoch_start_time = time.time()
        model.train()
        running_loss, correct, total = 0, 0, 0

        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

        train_loss = running_loss / len(train_loader)
        train_acc = 100 * correct / total

        # Validation pass
        model.eval()
        val_loss, val_correct, val_total = 0, 0, 0
        with torch.no_grad():
            for inputs, labels in val_loader:
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = model(inputs)
                loss = criterion(outputs, labels)
                val_loss += loss.item()
                _, predicted = torch.max(outputs.data, 1)
                val_total += labels.size(0)
                val_correct += (predicted == labels).sum().item()

        val_loss /= len(val_loader)
        val_acc = 100 * val_correct / val_total

        epoch_time = time.time() - epoch_start_time

        # Log epoch metrics to wandb
        wandb.log({
            'train_loss': train_loss,
            'train_accuracy': train_acc,
            'val_loss': val_loss,
            'val_accuracy': val_acc,
            'epoch_time': epoch_time
        })

        print(f"Epoch {epoch+1} | Time: {epoch_time:.2f}s | "
              f"Train Loss: {train_loss:.3f}, Train Acc: {train_acc:.2f}% | "
              f"Val Loss: {val_loss:.3f}, Val Acc: {val_acc:.2f}%")

    # Final test pass after all epochs
    model.eval()
    test_loss, test_correct, test_total = 0, 0, 0
    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            test_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            test_total += labels.size(0)
            test_correct += (predicted == labels).sum().item()

    test_loss /= len(test_loader)
    test_acc = 100 * test_correct / test_total

    # Log final test metrics
    wandb.log({
        'test_loss': test_loss,
        'test_accuracy': test_acc
    })

    print(f"\nFinal Test Accuracy: {test_acc:.2f}% | Final Test Loss: {test_loss:.3f}")

In [ ]:
wandb.finish()
sweep_id = wandb.sweep(sweep_config, project="cnn_sweep_project")
wandb.agent(sweep_id, function=train_sweep, count=50)  # Runs 50 experiments

Create sweep with ID: g6fvy9me
Sweep URL: https://wandb.ai/yeshu183-indian-institute-of-technology-madras/cnn_sweep_project/sweeps/g6fvy9me


wandb: Agent Starting Run: x6qvjxbp with config:
wandb: 	activation_func: Mish
wandb: 	batch_norm: True
wandb: 	batch_size: 64
wandb: 	data_augmentation: True
wandb: 	dropout: 0.3
wandb: 	m: 64
wandb: 	m_factor: 0.5


Epoch 1 | Time: 27.86s | Train Loss: 2.225, Train Acc: 16.44% | Val Loss: 2.184, Val Acc: 20.65%
Epoch 2 | Time: 27.70s | Train Loss: 2.180, Train Acc: 19.16% | Val Loss: 2.193, Val Acc: 21.20%
Epoch 3 | Time: 27.12s | Train Loss: 2.171, Train Acc: 19.90% | Val Loss: 2.145, Val Acc: 20.40%
Epoch 4 | Time: 27.36s | Train Loss: 2.153, Train Acc: 20.75% | Val Loss: 2.142, Val Acc: 23.45%
Epoch 5 | Time: 27.51s | Train Loss: 2.140, Train Acc: 21.35% | Val Loss: 2.152, Val Acc: 22.25%
Epoch 6 | Time: 27.40s | Train Loss: 2.132, Train Acc: 22.44% | Val Loss: 2.203, Val Acc: 19.85%
Epoch 7 | Time: 27.32s | Train Loss: 2.120, Train Acc: 22.59% | Val Loss: 2.138, Val Acc: 21.80%
Epoch 8 | Time: 27.28s | Train Loss: 2.110, Train Acc: 22.75% | Val Loss: 2.164, Val Acc: 22.00%
Epoch 9 | Time: 27.39s | Train Loss: 2.107, Train Acc: 23.19% | Val Loss: 2.114, Val Acc: 22.00%
Epoch 10 | Time: 27.49s | Train Loss: 2.092, Train Acc: 23.09% | Val Loss: 2.123, Val Acc: 22.05%
Epoch 11 | Time: 27.51s | Tra

epoch_time,█▇▁▃▅▄▃▃▄▄▅▄▃▃▃
test_accuracy,▁
test_loss,▁
train_accuracy,▁▃▄▄▅▆▆▆▆▆▇▇▇██
train_loss,█▆▆▅▄▄▄▃▃▂▂▂▂▁▁
val_accuracy,▃▄▂█▆▁▅▅▅▅▆▇██▅
val_loss,▇▇▄▄▅█▄▅▂▃▅▄▂▃▁
epoch_time,27.38161
test_accuracy,24.3
test_loss,2.09692
train_accuracy,25.24066


wandb: Agent Starting Run: g2bi0ypq with config:
wandb: 	activation_func: ReLU
wandb: 	batch_norm: False
wandb: 	batch_size: 32
wandb: 	data_augmentation: True
wandb: 	dropout: 0.3
wandb: 	m: 64
wandb: 	m_factor: 2


...

wandb: Agent Starting Run: n2e39f23 with config:
wandb: 	activation_func: SiLU
wandb: 	batch_norm: False
wandb: 	batch_size: 64
wandb: 	data_augmentation: False
wandb: 	dropout: 0.3
wandb: 	m: 32
wandb: 	m_factor: 1


Epoch 1 | Time: 12.38s | Train Loss: 2.305, Train Acc: 11.40% | Val Loss: 2.303, Val Acc: 11.15%
Epoch 2 | Time: 11.79s | Train Loss: 2.292, Train Acc: 11.76% | Val Loss: 2.269, Val Acc: 16.10%
Epoch 3 | Time: 12.77s | Train Loss: 2.268, Train Acc: 14.24% | Val Loss: 2.265, Val Acc: 14.60%
Epoch 4 | Time: 11.35s | Train Loss: 2.244, Train Acc: 15.26% | Val Loss: 2.263, Val Acc: 15.90%
Epoch 5 | Time: 11.79s | Train Loss: 2.221, Train Acc: 16.71% | Val Loss: 2.222, Val Acc: 18.00%
Epoch 6 | Time: 12.24s | Train Loss: 2.206, Train Acc: 17.65% | Val Loss: 2.192, Val Acc: 18.75%
Epoch 7 | Time: 12.32s | Train Loss: 2.192, Train Acc: 19.50% | Val Loss: 2.186, Val Acc: 20.70%
Epoch 8 | Time: 11.99s | Train Loss: 2.186, Train Acc: 18.38% | Val Loss: 2.192, Val Acc: 19.00%
Epoch 9 | Time: 11.56s | Train Loss: 2.179, Train Acc: 18.40% | Val Loss: 2.193, Val Acc: 19.00%
Epoch 10 | Time: 11.58s | Train Loss: 2.162, Train Acc: 19.74% | Val Loss: 2.192, Val Acc: 21.20%
Epoch 11 | Time: 11.91s | Tra

epoch_time,▆▃█▁▃▅▆▄▂▂▄▂▄▂▆
test_accuracy,▁
test_loss,▁
train_accuracy,▁▁▃▃▄▄▅▅▅▆▆▇▇▇█
train_loss,██▇▆▆▅▅▅▄▄▃▃▃▂▁
val_accuracy,▁▄▃▄▆▆▇▆▆█▇▇██▇
val_loss,█▆▆▆▄▂▂▂▂▂▄▃▁▂▄
epoch_time,12.44454
test_accuracy,21
test_loss,2.21195
train_accuracy,24.04051


wandb: Agent Starting Run: gqwudtqj with config:
wandb: 	activation_func: Mish
wandb: 	batch_norm: True
wandb: 	batch_size: 32
wandb: 	data_augmentation: True
wandb: 	dropout: 0.2
wandb: 	m: 64
wandb: 	m_factor: 1


Epoch 1 | Time: 29.45s | Train Loss: 2.264, Train Acc: 16.16% | Val Loss: 2.234, Val Acc: 17.65%
Epoch 2 | Time: 30.96s | Train Loss: 2.215, Train Acc: 18.25% | Val Loss: 2.249, Val Acc: 17.15%
Epoch 3 | Time: 28.95s | Train Loss: 2.206, Train Acc: 18.94% | Val Loss: 2.190, Val Acc: 20.95%


In [13]:
# import matplotlib.pyplot as plt

# epochs = range(1, len(train_losses) + 1)

# # Plot Loss
# plt.figure(figsize=(12, 5))
# plt.subplot(1, 2, 1)
# plt.plot(epochs, train_losses, label='Train Loss', marker='o')
# plt.plot(epochs, val_losses, label='Validation Loss', marker='o')
# plt.title('Loss over Epochs')
# plt.xlabel('Epoch')
# plt.ylabel('Loss')
# plt.legend()
# plt.grid(True)

# # Plot Accuracy
# plt.subplot(1, 2, 2)
# plt.plot(epochs, train_accuracies, label='Train Accuracy', marker='o')
# plt.plot(epochs, val_accuracies, label='Validation Accuracy', marker='o')
# plt.title('Accuracy over Epochs')
# plt.xlabel('Epoch')
# plt.ylabel('Accuracy (%)')
# plt.legend()
# plt.grid(True)

# plt.tight_layout()
# plt.show()